Data set is taken from here: https://dataverse.harvard.edu/dataset.xhtml?persistentId=hdl:1902.1/12379

In [52]:
# Import all the libraries used in the notebook
import pandas as pd
from matplotlib import pyplot as plt
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image
from IPython.core.display import HTML 
from sklearn.cluster import KMeans, k_means
from sklearn.metrics import silhouette_score
from sklearn.datasets.samples_generator import make_blobs
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
from sklearn.preprocessing import scale
%matplotlib inline
import hdbscan
from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from pprint import pprint

In [53]:
# read tab file of voting records (1948 - 2017)
df_votes = pd.read_csv('~/Documents/Capstone-UN/UNVotes1-72.tab',sep='\t')

In [54]:
# filter df for only voting members
df_votes = df_votes[df_votes.member == 1]

In [55]:
# Dropping country and countryname. Using ccode as country key
df_votes_year = df_votes.drop(['Country', 'Countryname'], axis=1) 

In [56]:
# Set a DF to only yes and no votes (0 nd 3)
df_votes_year_binary = df_votes_year[(df_votes_year.vote <= 1) | (df_votes_year.vote == 3)]


In [57]:
# Turn 1 = yes and 3 = no into 0 = no and 1 = yes
df_votes_year_binary.vote = df_votes_year_binary.vote.replace(to_replace=3, value=0, inplace=False, limit=None, regex=False, method='pad', axis=None)

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [58]:
# drop session column and set equal to new df
df_year_features = df_votes_year_binary.drop(['session'], axis=1)

In [59]:
# Vote multiplied by res to create a new v_res column that gives the resolution number if 
# There was a yes vote.  This will be used later to create dummy columns
df_year_features['v_res'] = df_year_features['rcid']*df_year_features['vote']

In [60]:
#I'm creating a df that has one record for every UN nation so that it can be merged with our output later
df_name_index = pd.DataFrame(df_votes.groupby(['ccode', 'Countryname','Country'])['Countryname'].count())
df_name_index['count'] = df_name_index.Countryname
df_name_index = df_name_index.drop([ 'Countryname','count'], axis=1)
df_name_index.reset_index(inplace=True)


# Create function to create DFs for multiple years

In [73]:
# This function takes a year as an argument and returns a df with 1s and 0s indicating whether each member country
# voted on each resolution proposed in a that year
def create_year_df(year,year2):
    df_output_year = df_year_features[df_year_features['year'].between(year, year2, inclusive=True)]
    #df_output_year = df_year_features[df_year_features['year'] == year]
    df_output_year = pd.concat([df_output_year,pd.get_dummies(df_output_year.v_res, prefix='res_')], axis = 1)
    #print(df_output_year)
    try:
        df_output_year = df_output_year.drop(['year','rcid','member','v_res','vote', 'res__0.0'], axis=1)
    except:
        pass 
    df_output_year = df_output_year.set_index('ccode')
    df_output_year = df_output_year.groupby('ccode').sum()
    
    return df_output_year


In [96]:
create_year_df(1999,2001).shape

(185, 202)

In [71]:
# read in descr file
df_descriptions = pd.read_csv('~/documents/UN-Capstone/Dscription.csv',skipinitialspace=True)
#THIS IS A FILE PATH TO A DIFFERENT FOLDER.  NEED TO FIX!

In [117]:

# read in descr file
df_GDP = pd.read_csv('~/documents/Capstone-UN/UNdata_Export_GDP copy.csv',skipinitialspace=True)
df_GDP.head(2)

,Countryname,Year,Item,GDP
0,Afghanistan,2016,Gross Domestic Product (GDP),583.882867
1,Afghanistan,2015,Gross Domestic Product (GDP),610.854517


## Output function

In [119]:
# This function forms the basis of the project's finished product and contrains code for prosessing the 
# UN vote DFs and; running through HDBSCAN to create lusters, printing the number of clusters, printing
# a silhouette score, printing a list of countries in each cluster, running the df through PCA analysis, 
# outputting a visualization of clusters based on principle components 1 and 2, and listing the resultions
# which caused the groupings (top 5 contriubuting the most to vaiance)


#HDBSCAN:
hdbscn = hdbscan.HDBSCAN(min_cluster_size = 35, min_samples =2, 
           gen_min_span_tree = True)
hdbscn.fit(create_year_df(1990,1990))
hdbscn_labels = hdbscn.labels_
hdbscn_n_clusters_ = len(set(hdbscn_labels)) - (1 if -1 in hdbscn_labels else 0)

df_output_year = create_year_df(1990,1990);
df_output_year['cluster'] = hdbscn.labels_
df_output_year.reset_index(inplace=True)
df_output_year = pd.merge(df_output_year, df_name_index, on='ccode')

#Create cluster lists
cluster0_list = df_output_year[df_output_year.cluster == 0]['Countryname'].tolist()
cluster1_list = df_output_year[df_output_year.cluster == 1]['Countryname'].tolist()
cluster2_list = df_output_year[df_output_year.cluster == 2]['Countryname'].tolist()
cluster_Outlier_list = df_output_year[df_output_year.cluster == -1]['Countryname'].tolist()

#PCA:
df_PCA = df_output_year.drop(['Countryname','Country'], axis=1)
df_features = df_PCA
X = df_PCA
X = X.set_index('ccode')
pca = PCA()
pca = pca.fit(X)
Z = pca.transform(X)
features_pca = ['PC'+str(i+1) for i in range(pca.n_components_)]

# Z = pd.DataFrame(Z, columns=features_pca)
# X.reset_index(inplace=True)
# X = X[['ccode','cluster']]

# X = X.join(Z, lsuffix='_caller', rsuffix='_other')
# df_PCA = pd.merge(X, df_PCA, on='ccode')
# df_PCA = df_PCA[['ccode','cluster_x',"PC1","PC2"]] 

# # Rolling Out PC1 components:
# PCA_abs = abs((pca.components_))[0]
# df_features = df_features.set_index(['ccode','cluster'])
# t = zip(df_features.columns, PCA_abs)
# result_list = list(t)
# df_componenets = pd.DataFrame(result_list)
# df_componenets.columns = ['rcid', 'score']
# df_componenets = df_componenets.sort_values('score',ascending=False)
# df_componenets['rcid'] = df_componenets['rcid'].map(lambda x: x.lstrip('res__'))
# df_componenets['rcid'] = df_componenets['rcid'].astype(float)

# df_componenets = pd.merge(df_componenets, df_descriptions, on='rcid')
# df_pca_desc = df_componenets.descr
# df_componenets = df_componenets.drop(['abstain','session','date',
#                                       'importantvote','unres','amend',
#                                       'para', 'year','Unnamed: 0','descr','me', 'nu','di', 'hr', 'co','ec',], axis=1) #,'short'

# pd.set_option('display.max_colwidth',110)

# #Plot PCA
# fig = plt.figure(figsize=(12,8))
# ax = fig.gca()
# ax.scatter(Z.iloc[:,0], Z.iloc[:,1],
#                    c=df_PCA.cluster_x)
# ax.set_xlabel('principal component 1', fontsize=16)
# ax.set_ylabel('principal component 2', fontsize=16)
# for tick in ax.xaxis.get_major_ticks():
#     tick.label.set_fontsize(12) 
# for tick in ax.yaxis.get_major_ticks():
#     tick.label.set_fontsize(12) 

# pd.set_option('display.max_colwidth', -1)   
# ax.set_title('PC1 vs PC2\n', fontsize=20)


# print HDBSCAN RESULTS:

print('Estimated number of clusters: %d' % hdbscn_n_clusters_)
# print('Homogeneity: %0.6f' % metrics.homogeneity_score(frames.get(year), hdbscn_labels))
# print('Completeness: %0.6f' % metrics.completeness_score(y, hdbscn_labels))
print('Silhouette Coefficient: %0.6f' % metrics.silhouette_score(create_year_df(1990,2016), hdbscn_labels))
#print(plt.show())

print ("\n\n","Cluster 0:",len(cluster0_list),"\n",cluster0_list)
print ("\n\n","Cluster 1:",len(cluster1_list),"\n",cluster1_list)
if len(cluster2_list) > 0:
    print ("\n\n","Cluster 2:",len(cluster2_list),"\n",cluster2_list)
else: ""

print ("\n\n","Outliers:",len(cluster_Outlier_list),"\n",cluster_Outlier_list)

#print('\n',df_componenets.head(10))
#return df_pca_desc.head(10)

  

Estimated number of clusters: 2
Silhouette Coefficient: 0.235468


 Cluster 0: 60 
 ['United States of America', 'Canada', 'United Kingdom of Great Britain and Northern Ireland', 'Ireland', 'Netherlands', 'Belgium', 'Luxembourg', 'France', 'Monaco', 'Liechtenstein', 'Switzerland', 'Spain', 'Andorra', 'Portugal', 'Germany', 'Poland', 'Austria', 'Hungary', 'Czechoslovakia', 'Czech Republic', 'Slovakia', 'Italy', 'San Marino', 'Malta', 'Albania', 'Montenegro', 'The former Yugoslav Republic of Macedonia', 'Croatia', 'Yugoslavia', 'Bosnia and Herzegovina', 'Slovenia', 'Greece', 'Cyprus', 'Bulgaria', 'Republic of Moldova', 'Romania', 'Estonia', 'Latvia', 'Lithuania', 'Ukraine', 'Armenia', 'Georgia', 'Finland', 'Sweden', 'Norway', 'Denmark', 'Iceland', 'South Sudan', 'Turkey', 'Israel', 'Republic of Korea', 'Japan', 'Timor-Leste', 'Australia', 'New Zealand', 'Kiribati', 'Tuvalu', 'Marshall Islands', 'Palau', 'Micronesia (Federated States of)']


 Cluster 1: 97 
 ['Bahamas', 'Cuba', 'Haiti', '

In [118]:
df_output_year.shape

(194, 1956)

In [122]:
df_output_wGDP = pd.merge(df_GDP,df_output_year, on='Countryname')

In [123]:
df_output_wGDP

,Countryname,Year,Item,GDP,ccode,res__3501.0,res__3502.0,res__3503.0,res__3504.0,res__3505.0,...,res__5534.0,res__5535.0,res__5536.0,res__5537.0,res__5538.0,res__5539.0,res__5540.0,res__5545.0,cluster,Country
0,Afghanistan,2016,Gross Domestic Product (GDP),583.882867,700,1,1,1,1,1,...,1,1,1,1,1,1,1,0,1,AFG
1,Afghanistan,2015,Gross Domestic Product (GDP),610.854517,700,1,1,1,1,1,...,1,1,1,1,1,1,1,0,1,AFG
2,Afghanistan,2014,Gross Domestic Product (GDP),651.158326,700,1,1,1,1,1,...,1,1,1,1,1,1,1,0,1,AFG
3,Afghanistan,2013,Gross Domestic Product (GDP),681.033974,700,1,1,1,1,1,...,1,1,1,1,1,1,1,0,1,AFG
4,Afghanistan,2012,Gross Domestic Product (GDP),694.885886,700,1,1,1,1,1,...,1,1,1,1,1,1,1,0,1,AFG
5,Afghanistan,2011,Gross Domestic Product (GDP),645.283587,700,1,1,1,1,1,...,1,1,1,1,1,1,1,0,1,AFG
6,Afghanistan,2010,Gross Domestic Product (GDP),558.199767,700,1,1,1,1,1,...,1,1,1,1,1,1,1,0,1,AFG
7,Afghanistan,2009,Gross Domestic Product (GDP),448.952721,700,1,1,1,1,1,...,1,1,1,1,1,1,1,0,1,AFG
8,Afghanistan,2008,Gross Domestic Product (GDP),394.536514,700,1,1,1,1,1,...,1,1,1,1,1,1,1,0,1,AFG
9,Afghanistan,2007,Gross Domestic Product (GDP),389.985428,700,1,1,1,1,1,...,1,1,1,1,1,1,1,0,1,AFG
